In [11]:
!pip uninstall optimum-neuron -y

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Found existing installation: optimum-neuron 0.1.1.dev0
Uninstalling optimum-neuron-0.1.1.dev0:
  Successfully uninstalled optimum-neuron-0.1.1.dev0


In [19]:
import optimum.neuron 

optimum.neuron.version.__version__

'0.1.1.dev0'

In [17]:
!transformers-cli env

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

Copy-and-paste the text below in your GitHub issue and FILL OUT the two last points.

- `transformers` version: 4.28.1
- Platform: Linux-5.15.0-1030-aws-x86_64-with-glibc2.29
- Python version: 3.8.10
- Huggingface_hub version: 0.13.3
- Safetensors version: not installed
- PyTorch version (GPU?): 1.13.1+cu117 (False)
- Tensorflow version (GPU?): not installed (NA)
- Flax version (CPU?/GPU?/TPU?): not installed (NA)
- Jax version: not installed
- JaxLib version: not installed
- Using GPU in script?: <fill in>
- Using distributed or parallel set-up in script?: <fill in>



In [9]:
!pip install transformers --upgrade

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 7.0 MB 40.6 MB/s eta 0:00:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.28.0
    Uninstalling transformers-4.28.0:
      Successfully uninstalled transformers-4.28.0


In [12]:
!pip install git+https://github.com/huggingface/optimum-neuron.git@fix_multiclass_training_issue

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Cloning https://github.com/huggingface/optimum-neuron.git (to revision fix_multiclass_training_issue) to /tmp/pip-req-build-0s_1d2jo
  Running command git clone -q https://github.com/huggingface/optimum-neuron.git /tmp/pip-req-build-0s_1d2jo
  Running command git checkout -b fix_multiclass_training_issue --track origin/fix_multiclass_training_issue
  Switched to a new branch 'fix_multiclass_training_issue'
  Branch 'fix_multiclass_training_issue' set up to track remote branch 'fix_multiclass_training_issue' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for 

In [13]:
!neuron-ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
instance-type: trn1.2xlarge
instance-id: i-0570615e41700a481
+--------+--------+--------+---------+
| NEURON | NEURON | NEURON |   PCI   |
| DEVICE | CORES  | MEMORY |   BDF   |
+--------+--------+--------+---------+
| 0      | 2      | 32 GB  | 00:1e.0 |
+--------+--------+--------+---------+


## Preprocess dataset

In [14]:
from datasets import load_dataset

# Dataset id from huggingface.co/dataset
dataset_id = "banking77"
# Model id to load the tokenizer
model_id = "bert-base-uncased"
save_dataset_path = "dataset"


# Load raw dataset
raw_dataset = load_dataset(dataset_id)

print(f"Train dataset size: {len(raw_dataset['train'])}")
print(f"Test dataset size: {len(raw_dataset['test'])}")

# Train dataset size: 10003
# Test dataset size: 3080
from random import randrange

random_id = randrange(len(raw_dataset['train']))
print(raw_dataset['train'][random_id])
# {'text': 'How can I change my PIN without going to the bank?', 'label': 21}

import os
from transformers import AutoTokenizer


# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Tokenize helper function
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True,return_tensors="pt")

# Tokenize dataset
raw_dataset =  raw_dataset.rename_column("label", "labels") # to match Trainer
tokenized_dataset = raw_dataset.map(tokenize, batched=True, remove_columns=["text"])
tokenized_dataset = tokenized_dataset.with_format("torch")

print(tokenized_dataset["train"].features.keys())
# dict_keys(['input_ids', 'token_type_ids', 'attention_mask','lable'])
# save dataset to disk
tokenized_dataset["train"].save_to_disk(os.path.join(save_dataset_path,"train"))
tokenized_dataset["test"].save_to_disk(os.path.join(save_dataset_path,"eval"))

/home/ubuntu/.cache/huggingface/modules/datasets_modules/datasets/banking77/9898c11f6afa9521953d2ef205667b527bad14ef9cab445d470f16240c8c8ec4/banking77.py:59: FutureWarning: Dataset 'banking77' is deprecated and will be deleted. Use 'PolyAI/banking77' instead.
  warnings.warn(
Found cached dataset banking77 (/home/ubuntu/.cache/huggingface/datasets/banking77/default/1.1.0/9898c11f6afa9521953d2ef205667b527bad14ef9cab445d470f16240c8c8ec4)
100%|██████████| 2/2 [00:00<00:00, 665.76it/s]
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/banking77/default/1.1.0/9898c11f6afa9521953d2ef205667b527bad14ef9cab445d470f16240c8c8ec4/cache-c98c758d88fafd27.arrow


Train dataset size: 10003
Test dataset size: 3080
{'text': 'Which merchants accept this card?', 'label': 10}


dict_keys(['labels', 'input_ids', 'token_type_ids', 'attention_mask'])


## precompiple 

In [3]:
!neuron_parallel_compile python3 scripts/train.py --model_id bert-base-uncased --per_device_train_batch_size 8 

2023-03-20 12:50:18.000261: INFO ||PARALLEL_COMPILE||: Running trial run (add option to terminate trial run early; also ignore trial run's generated outputs, i.e. loss, checkpoints)
is precompilation: 1
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly

`2023-03-13 08:24:52.000851: ERROR ||PARALLEL_COMPILE||: parallel compilation with neuronx-cc exited with error.Received error code: 137`

=> OOM? but training with BS 8 works. 

```
2023-03-13 08:26:28.000757: INFO ||PARALLEL_COMPILE||: Total graphs: 2
2023-03-13 08:26:28.000757: INFO ||PARALLEL_COMPILE||: Total successful compilations: 0
2023-03-13 08:26:28.000757: INFO ||PARALLEL_COMPILE||: Total failed compilations: 2
```

monitoring `htop` has shown that we ran out of CPU Memory -> (30.8GB)

`free -h`

shows no swap 


```bash
# add swap 10GB of swap
sudo fallocate -l 20G /swapfile
# change permissions
sudo chmod 600 /swapfile
# mark the file as swap space
sudo mkswap /swapfile
# enable swap 
sudo swapon /swapfile
# make swap file permanent
# echo '/swapfile none swap sw 0 0' | sudo tee -a /etc/fstab
```

20 GB was not enough! 🤯


![img](./im.png)

switch swap: https://askubuntu.com/questions/1264568/increase-swap-in-20-04

# train

In [17]:
!rm -r /var/tmp/neuron-compile-cache

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [6]:
pip install huggingface_hub --upgrade

     |████████████████████████████████| 199 kB 39.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import huggingface_hub

huggingface_hub.__version__

'0.12.1'

In [29]:
!CUSTOM_CACHE_REPO=philschmid/neuron-cache python3 scripts/train.py  --model_id bert-base-cased --per_device_train_batch_size 8 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
is precompilation: None
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining mo

In [30]:
!CUSTOM_CACHE_REPO=philschmid/neuron-cache python3 scripts/run_glue.py  --model_name_or_path bert-base-cased --per_device_train_batch_size 8 --do_train True --do_eval True --dataset_name banking77 --evaluation_strategy "epoch" --output_dir ./run23434

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/ubuntu/.cache/huggingface/modules/datasets_modules/datasets/banking77/9898c11f6afa9521953d2ef205667b527bad14ef9cab445d470f16240c8c8ec4/banking77.py:59: FutureWarning: Dataset 'banking77' is deprecated and will be deleted. Use 'PolyAI/banking77' instead.
  warnings.warn(
100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 697.13it/s]
[WARNING|modeling_utils.py:3180] 2023-04-19 18:38:49,220 >> Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weig

# Distrubted test 

In [15]:
!rm -r /var/tmp/neuron-compile-cache

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [32]:
!CUSTOM_CACHE_REPO=philschmid/neuron-cache torchrun --nproc_per_node=2 scripts/run_glue.py  --model_name_or_path bert-base-cased --per_device_train_batch_size 8 --do_train True --do_eval True --bf16 True --dataset_name banking77 --evaluation_strategy "epoch" --output_dir ./bf16

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
/home/ubuntu/.cache/huggingface/modules/datasets_modules/datasets/banking77/9898c11f6afa9521953d2ef205667b527bad14ef9cab445d470f16240c8c8ec4/banking77.py:59: FutureWarning: Dataset 'banking77' is deprecated and will be deleted. Use 'PolyAI/banking77' instead.
  warnings.warn(
100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 690.36it/s]
/home/ubuntu/.cache/hugg

In [35]:
!CUSTOM_CACHE_REPO=philschmid/neuron-cache torchrun --nproc_per_node=2 scripts/train.py --model_id bert-base-uncased --lr 3e-5 --per_device_train_batch_size 8

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
is precompilation: None
is precompilation: None
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias

1760.8812s -> 591.1733s 


bf16 {'train_runtime': 426.354, 'train_samples_per_second': 70.385, 'train_steps_per_second': 4.405, 'train_loss': 4.362087326943557, 'epoch': 3.0}